In [ ]:
# Plotly and Dash library
!pip install dash
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import plotly.express as px
import plotly

# Arrays and Dataframe
from collections import deque  # An array that can contain a certain amount of values
                               # it deletes automatically the oldest
import pandas as pd
import numpy as np

# Other
import random
import json
import ast

# Custom
from dashboard_utils import *

In [ ]:
def maxfigures(tab):
    allmax = 0
    for i in range(4):
        thismax = max(figures[tab][i])
        
        if thismax > allmax:
            allmax = thismax
    
    return allmax

In [ ]:
figure1 = [[],[],[],[]]
figure2 = [[],[],[],[]]
figure3 = [[],[],[],[]]
figure4 = [[],[],[],[]]
bins1 = [[],[],[],[]]
bins2 = [[],[],[],[]]
bins3 = [[],[],[],[]]
bins4 = [[],[],[],[]]

bins5 = [[],[],[],[]]
figure5 = [[0]*200,[0]*200,[0]*200,[0]*200]

figures = [figure1,figure2,figure3,figure4,figure5]
bins = [bins1,bins2,bins3,bins4,bins5]

tabledata = 0
tabledata_chamber = [0,0,0,0]



In [ ]:
'''Gets the data from file and creates'''
def counttable_getdata():
    # Update the table from a dataframes using dictionaries
    df = pd.DataFrame()
    data = {'ch': 'All', 'counts': tabledata}
    df = df.append(data, ignore_index=True)
    for i in range(4):
        data = {'ch': 'Chamber '+str(i), 'counts':tabledata_chamber[i]}
        df = df.append(data, ignore_index=True)
    
    return df.to_dict('records')

''' Load from the files the necessary data to build the live histogram'''
def hist_getdata(numhist,chamber):
    global bins
    global figures
    
    # Load the data from the dictionary
    data = {'y': figures[numhist-1][chamber-1],
            'x': bins[numhist-1][chamber-1],
            'type':'bar'}
    
    return {'data': [data], 'layout' : go.Layout(title='Chamber '+str(chamber - 1),
                                                 yaxis=dict(title = titleaxes[numhist-1][1],
                                                            range=[0,maxfigures(numhist-1)]),
                                                 xaxis=dict(title = titleaxes[numhist-1][0],
                                                            range=[min(bins[numhist-1][chamber-1]),
                                                                   max(bins[numhist-1][chamber-1])]))}

In [ ]:
# Stuff for the plot (UP LEFT)
index = [0]

totalcount_X = deque(maxlen=10)
totalcount_X.append(0)
totalcount = deque(maxlen=10)
totalcount.append(0)

In [ ]:
'''TABS'''
# General
tabs_styles = {'height': '44px'}

# The tab
tab_style = {
                'borderBottom': '1px solid #d6d6d6',
                'padding': '6px',
                'fontWeight': 'bold'
            }

# Extra tab
extra_tab_style1 = {
                        'borderBottom': '1px solid #FFDC00 ',
                        'borderTop': '1px solid #FFDC00 ',
                        'borderLeft': '1px solid #FFDC00 ',
                        'padding': '6px',
                        'fontWeight': 'bold'
                   }

extra_tab_style2 = {
                        'borderBottom': '1px solid #FFDC00 ',
                        'borderTop': '1px solid #FFDC00 ',
                        'borderRight': '1px solid #FFDC00 ',
                        'padding': '6px',
                        'fontWeight': 'bold'
                   }

extra_tab_style3 = {
                        'borderBottom': '1px solid #FFDC00 ',
                        'borderTop': '1px solid #FFDC00 ',
                        'padding': '6px',
                        'fontWeight': 'bold'
                   }

# Tab when active
tab_selected_style = {'padding': '6px','fontWeight': 'bold'}

extra_tab_selected_style = {'padding': '6px','fontWeight': 'bold','borderTop': '2px solid #FFDC00 '}

# Columns name (for creating the dataframe and stuff)
tblcols=[{'name': 'Chamber', 'id': 'ch'},
         {'name': 'Counts', 'id': 'counts'}]

'''HISTOGRAMS''' 
# Container
hist_div_style = {'display': 'inline-block', 'width':'50%'}

# The histogram
hist_style = {'height':'20vw'}

'''TABLE'''
# Header
table_header_style = {'fontWeight': 'default'}

# The table
table_style = {'width': '180%'}

# Cells
table_cell_style = {'textAlign': 'left', 'fontSize': '120%'}

# Conditional [if the column is the first, we bold it]
table_conditional_style = [{
                            'if': {'column_id': 'ch'},
                            'fontWeight': 'bold'
                          }]

'''OTHER'''
titleaxes = [['TDC_CHANNEL','Counts'],
             ['Active TDC_CHANNEL','ORBIT_CNT'],
             ['Active TDC_CHANNEL','Scint. ORBIT_CNT'],
             ['DRIFTIME','Counts'],
             ['DRIFTIME','Counts']]

In [ ]:
app = dash.Dash(__name__)

# LAYOUT OF THE WEB PAGE
app.layout = html.Div(
    [   # TITLE
        html.H2('Live Dashboard 🔴'),
        # to make it side by side you need both elements to be in children=[,] and have style={'display': 'inline-block'}
        html.Div(children=[
            # Container of Graph
            html.Div(dcc.Graph(id='live-graph', animate=True),
                     style={'width': '60%', 'display': 'inline-block',"margin-right": "5%"}),
            # Container of Table
            html.Div(dash_table.DataTable(id = 'count_table', data = counttable_getdata(), columns=tblcols,
                                          style_header=
                                            {
                                                'fontWeight': 'default'
                                            },
                                          style_table={'width': '180%'},
                                          style_cell={'textAlign': 'left', 'fontSize': '120%'},
                                          style_cell_conditional=[
                                            {
                                                'if': {'column_id': 'ch'},
                                                'fontWeight': 'bold'
                                            }]),style={'display': 'inline-block',"verticalAlign": "top",
                                                       "margin-top": "5%"})
            
        ]),
        html.Div([
            dcc.Tabs(id='tabs', value='tab-1', children=[
                    dcc.Tab(label='Histogram 1', value='tab-1', style=tab_style, selected_style=tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist1-1', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-2', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-3', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist1-4', animate=True,
                                                style=hist_style),style=hist_div_style)
                                     ]),
                    dcc.Tab(label='Histogram 2', value='tab-2', style=tab_style, selected_style=tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist2-1', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-2', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-3', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist2-4', animate=True,
                                                style=hist_style),style=hist_div_style)
                                    ]),
                    dcc.Tab(label='EXTRA 1', value='tab-3', style=extra_tab_style1, selected_style=extra_tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist3-1', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist3-2', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist3-3', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist3-4', animate=True,
                                                style=hist_style),style=hist_div_style)
                                    ]),
                
                    dcc.Tab(label='EXTRA 2', value='tab-4', style=extra_tab_style3, selected_style=extra_tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist4-1', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist4-2', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist4-3', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist4-4', animate=True,
                                                style=hist_style),style=hist_div_style)
                                    ]),
                    
                    dcc.Tab(label='EXTRA 2 (cumulative)', value='tab-5', style=extra_tab_style2, selected_style=extra_tab_selected_style,
                            children=[html.Div(dcc.Graph(id = 'hist5-1', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist5-2', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist5-3', animate=True,
                                                style=hist_style),style=hist_div_style),
                                      html.Div(dcc.Graph(id = 'hist5-4', animate=True,
                                                style=hist_style),style=hist_div_style)
                                    ])
                ]),
                    
        ]),
    dcc.Interval(id='graph-update')]
)

# CALLBACK FOR COUNT GRAPH
@app.callback(Output('live-graph', 'figure'),
        [Input('graph-update', 'n_intervals')])
def update_graph_scatter(n):
    global tabledata
    global tabledata_chamber
    global figures
    global bins
    
    message = load_obj('message')
    if message['Index'] != index[-1]:
        if len(totalcount_X) != 0:
            totalcount_X.append(totalcount_X[-1] + 1)
        else:
            totalcount_X.append(0)
            
        totalcount.append(message['Total Count'])
        
        index.append(message['Index'])
        
        # TABLE UPDATE
        tabledata = totalcount[-1]
        for i in range(4):
            tabledata_chamber[i] = message['Chamber_'+str(i+1)]['Count']
            
        # FIGURES
        for tab in range(4):
            for chamber in range(4):
                figures[tab][chamber] = message['Chamber_'+str(chamber+1)]['Hist_'+str(tab+1)]['Counts']
                bins[tab][chamber] = message['Chamber_'+str(chamber+1)]['Hist_'+str(tab+1)]['Bins'][1:]
                
        for chamber in range(4):
            bins[4][chamber] = bins[3][chamber]
            figures[4][chamber] = [sum(x) for x in zip(figures[4][chamber], figures[3][chamber])]
    # UPDATE THE SCATTER        
    data = plotly.graph_objs.Scatter(
            x=list(totalcount_X),
            y=list(totalcount),
            name='Scatter',
            mode= 'lines+markers'
            )
    if len(totalcount_X) == 1:
        return {'data': [data],'layout' : go.Layout(height=300,
                                                    title='Total Count of Events')}
    else:
        return {'data': [data],'layout' : go.Layout(height=300,
                                                    title='Total Count of Events',
                                                    xaxis=dict(title='Time step',
                                                               range=[min(totalcount_X),max(totalcount_X)]),
                                                    yaxis=dict(title='Counts',
                                                               range=[min(totalcount),max(totalcount)]),)}

# CALLBACK FOR TABLE
@app.callback(Output('count_table','data'),
             [Input('graph-update', 'n_intervals')])
def updateTable(n):
     return counttable_getdata()
    
# CALLBACK FOR HIST

### HIST 1 ###
@app.callback(Output('hist1-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist1(n):
     return hist_getdata(1,1)
    
@app.callback(Output('hist1-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist2(n):
     return hist_getdata(1,2)
    
@app.callback(Output('hist1-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist3(n):
     return hist_getdata(1,3)
    
@app.callback(Output('hist1-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist4(n):
     return hist_getdata(1,4)
@app.callback(Output('hist2-1','figure'),
             [Input('graph-update', 'n_intervals')])

### HIST 2 ###
def updateHist5(n):
     return hist_getdata(2,1)
    
@app.callback(Output('hist2-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist6(n):
     return hist_getdata(2,2)
    
@app.callback(Output('hist2-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist7(n):
     return hist_getdata(2,3)
    
@app.callback(Output('hist2-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist8(n):
     return hist_getdata(2,4)

### EXTRA 1 ###
@app.callback(Output('hist3-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist9(n):
     return hist_getdata(3,1)
    
@app.callback(Output('hist3-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist10(n):
     return hist_getdata(3,2)
    
@app.callback(Output('hist3-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist11(n):
     return hist_getdata(3,3)
    
@app.callback(Output('hist3-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist12(n):
     return hist_getdata(3,4)


### EXTRA 2 ###
@app.callback(Output('hist4-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist13(n):
     return hist_getdata(4,1)
    
@app.callback(Output('hist4-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist14(n):
     return hist_getdata(4,2)
    
@app.callback(Output('hist4-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist15(n):
     return hist_getdata(4,3)
    
@app.callback(Output('hist4-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist16(n):
     return hist_getdata(4,4)
    
### EXTRA 2 (CUMULATIVE) ###
@app.callback(Output('hist5-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist13(n):
     return hist_getdata(5,1)
    
@app.callback(Output('hist5-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist14(n):
     return hist_getdata(5,2)
    
@app.callback(Output('hist5-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist15(n):
     return hist_getdata(5,3)
    
@app.callback(Output('hist5-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist16(n):
     return hist_getdata(5,4)
    
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)
